# Init


In [ ]:
# Install necessary requirements
%pip install -r ../requirements.txt

In [ ]:
# Change working directory to root
import os
if os.getcwd().endswith("notebooks"):
    %cd ..
    print(os.getcwd())

# Automatically reload changes in code
%load_ext autoreload
%autoreload 2

In [ ]:
import logging
from typing import List
import pandas as pd
from association_finder.concept_drifts_finder import ConceptDriftsFinder
from association_finder.models import Transaction, ConceptDriftResult
from association_finder.concept_engineering import ConceptEngineering
from sklearn.model_selection import train_test_split
from association_finder.preprocessing import preprocess_dataset, split_X_y
from typing import Dict, Tuple, Optional
from dataclasses import dataclass
import numpy as np
from sklearn.linear_model import LogisticRegression

# Logs config
logging.basicConfig(level=logging.INFO)

# Pandas config
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)

# Read and parse file

In [ ]:
np.random.seed(0)

# Read file
train_dataset_path = "datasets/big_mart_sales/Train-Set.csv"
df = pd.read_csv(train_dataset_path, index_col='ProductID')
target_column = "OutletSales"

# Drop rows with NaN values in the target column.
df.drop(df[df[target_column].isna()].index,inplace=True)

# Split
df_train, df_val = train_test_split(df, test_size=0.3)

# Preprocess    
df_train_prep, train_params = preprocess_dataset(df_train)

# Focusing on prominent columns:
good_columns = [column for column in ['Weight', 'FatContent', 'ProductVisibility', 'ProductType', 'MRP',
                'OutletID', 'EstablishmentYear', 'OutletSize', 'LocationType', 'OutletType', 'OutletSales']
                if column not in train_params.dropped_columns]
one_hot_columns = [column for column in ['FatContent','ProductType','OutletID','OutletSize','LocationType','OutletType']
                   if column not in train_params.dropped_columns]

In [ ]:
# Prepare data for training
X_train, y_train = split_X_y(df_train_prep, good_columns, train_params, one_hot_columns, target_column)
X_val, y_val = split_X_y(preprocess_dataset(df_val, train_params)[0], good_columns, train_params, one_hot_columns, target_column)

# Find rules and analyze them

In [ ]:
# Find association rules
concept_engineering = ConceptEngineering(verbose=True, min_confidence=0.1, min_support=0.1, diff_threshold=0.1, concept_drift_size_threshold=10)
X_train_rules = concept_engineering.fit_transform(X_train, df_train_prep[good_columns], target_column, one_hot_columns)
X_val_rules = concept_engineering.transform(X_val)

In [ ]:
concept_engineering.concepts_df

# TODO: No rules found!

# Baseline

In [ ]:
# Train baseline model
clf = LogisticRegression(random_state=0, max_iter=10000).fit(X_train, y_train)

In [ ]:
print(f"Train accuracy: {clf.score(X_train, y_train)}")
print(f"Validation accuracy: {clf.score(X_val, y_val)}")

# Expected:
# Train accuracy: 0.500167616493463
# Validation accuracy: 0.49706687524442705

# Build model using rules

In [ ]:
# Train model using association rules
clf_rules = LogisticRegression(random_state=0, max_iter=10000).fit(X_train_rules, y_train)

In [ ]:
print(f"Train accuracy: {clf_rules.score(X_train_rules, y_train)}")
print(f"Validation accuracy: {clf_rules.score(X_val_rules, y_val)}")

# TODO: Same accuracy before and after the rules.

# Analyze concepts